In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

from ccb.experiment import parse_results
from ccb.experiment.retrieve_runs import retrieve_runs

%load_ext autoreload
%autoreload 2

# RGB Classification

In [ ]:
# directory where all the sweeps experiments are logged
classification_dir =  '/mnt/data/experiments/nils/last_classification_sweeps'
classification_dir = '/mnt/data/experiments/nils/final_classification_seeded_runs'

df = retrieve_runs(classification_dir, use_cached_csv=False, is_sweep=False)

In [ ]:
# generate best hparam config from sweeps for seeded runs
best_runs = parse_results.find_best_hparam_for_seeds(df)

In [ ]:

df_1x = df[(df["partition_name"] == "1.00x_train") | (df["partition_name"] == "default")]

best_points_df = parse_results.plot_all_models_datasets(df_1x, plot_fn=parse_results.make_plot_sweep(filt_size=10, top_k=6, legend=False), fig_size=(60,40))
plt.tight_layout()


In [ ]:

# plt.rcParams['figure.figsize'] = [40, 40]

parse_results.plot_all_datasets(df_1x, model="vit_tiny_patch16_224", plot_fn=parse_results.make_plot_sweep(filt_size=20, top_k=10, legend=True), fig_size=(20,20))
# parse_results.plot_all_datasets(df_1x, model="resnet50")

plt.tight_layout()


In [ ]:
parse_results.plot_all_models_datasets(df_1x, plot_fn=parse_results.make_plot_sweep(filt_size=20, top_k=6, legend=False), fig_size=(40,40))
plt.tight_layout()

# Beyond RGB Classification

In [ ]:
# directory where all the sweeps experiments are logged
beyond_classification_dir =  '/mnt/data/experiments/nils/beyond_rgb_classification_sweeps'
beyond_classification_dir = '/mnt/data/experiments/nils/final_classification_beyond_rgb_seeded_runs'

beyond_rgb_models_df = retrieve_runs(beyond_classification_dir, use_cached_csv=False, is_sweep=False)

In [ ]:
best_points_df = parse_results.plot_all_models_datasets(beyond_rgb_models_df, plot_fn=parse_results.make_plot_sweep(filt_size=10, top_k=6, legend=False), fig_size=(40,40))
plt.tight_layout()

# RGB Segmentation

In [ ]:
# directory where all the sweeps experiments are logged
segmentation_dir =  '/mnt/data/experiments/nils/last_segmentation_sweeps'
segmentation_dir = '/mnt/data/experiments/nils/final_segmentation_seeded_runs'
df_seg = retrieve_runs(segmentation_dir, use_cached_csv=False, is_sweep=True)

In [ ]:

df_1x_seg = df_seg[(df_seg["partition_name"] == "1.00x_train") | (df_seg["partition_name"] == "default")]

best_points_df = parse_results.plot_all_models_datasets(df_1x_seg, plot_fn=parse_results.make_plot_sweep(filt_size=10, top_k=6, legend=False), fig_size=(40,40))
plt.tight_layout()

In [ ]:
parse_results.count_exp(df_seg)